In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Activation, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import plot_confusion_matrix, precision_score, recall_score, f1_score
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.tree import DecisionTreeClassifier
import math
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [ ]:
data = pd.read_csv('/kaggle/input/migraine-classification/data.csv')
df = pd.read_csv('/kaggle/input/migraine-classification/data.csv')

In [ ]:
data.shape

In [ ]:
print('Number of training examples: %s' %str (data.shape [0]))
print('Number of features: %s' %str (data.shape [1]))

In [ ]:
data.columns

In [ ]:
data['Type'].value_counts()

In [ ]:
data.Age.min() , data.Age.max()

# ***Visualisation***

In [ ]:
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", annot_kws={"size": 8})
import matplotlib.pyplot as plt
plt.show()

In [ ]:
plt.title('Range of Patient Age')
data['Age'].hist()

In [ ]:
data.Character.value_counts()

In [ ]:
sns.countplot(x=data['Character'])

In [ ]:
sns.countplot(x=data.Defect)

In [ ]:
data.Type.value_counts()

In [ ]:
sns.set(rc={'figure.figsize':(15, 10)})
sns.countplot(x=data.Type)

In [ ]:
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(data.Type.value_counts(), labels=['Typical Aura With Migraine', 'Migraine Without Aura', 'Familial Hemiplegic Nigraine',
                                                'Typical Aura Without Migraine', 'Basilar Typed Aura', 
                                         'Other', 'Sporadic Hemiplegic Migraine'], colors=['blue', 'magenta', 'cyan', 
                                                                                   'red', 'grey', 'black', 'yellow'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:

sns.catplot(y='Type', x='Age', data=data.sort_values("Type", ascending=False),
           kind='boxen', height=6, aspect=3)

In [ ]:
data.isnull().sum().to_numpy()

In [ ]:
#dataframes

In [ ]:
data_1 = data[data['Type'] == 'Typical aura with migraine']
data_2 = data[data['Type'] == 'Migraine without aura']
data_3 = data[data['Type'] == 'Familial hemiplegic migraine']
data_4 = data[data['Type'] == 'Typical aura without migraine']
data_5 = data[data['Type'] == 'Basilar-type aura']
data_6 = data[data['Type'] == 'Other']
data_7 = data[data['Type'] == 'Sporadic hemiplegic migraine']

In [ ]:
data.shape

In [ ]:
400/7

In [ ]:
from sklearn.utils import resample
data_1_resample = resample(data_1, n_samples=200, 
                           random_state=123, replace=True)
data_2_resample = resample(data_2, n_samples=200, 
                           random_state=123, replace=True)
data_3_resample = resample(data_3, n_samples=200, 
                           random_state=123, replace=True)
data_4_resample = resample(data_4, n_samples=200, 
                           random_state=123, replace=True)
data_5_resample = resample(data_5, n_samples=200, 
                           random_state=123, replace=True)
data_6_resample = resample(data_6, n_samples=200, 
                           random_state=123, replace=True)
data_7_resample = resample(data_7, n_samples=200, 
                           random_state=123, replace=True)

In [ ]:
train_dataset = pd.concat([data_1, data_2_resample, data_3_resample, data_4_resample, 
                          data_5_resample, data_6_resample, data_7_resample])

In [ ]:
train_dataset.head(3)

In [ ]:
train_dataset.shape

In [ ]:
plt.figure(figsize=(10, 8))
circle = plt.Circle((0, 0), 0.7, color='white')
plt.pie(train_dataset.Type.value_counts(), labels=['Typical Aura With Migraine', 'Migraine Without Aura', 'Familial Hemiplegic Nigraine',
                                                'Typical Aura Without Migraine', 'Basilar Typed Aura', 
                                         'Other', 'Sporadic Hemiplegic Migraine'], colors=['blue', 'magenta', 'cyan', 
                                                                                   'red', 'grey', 'black', 'yellow'])
p = plt.gcf()
p.gca().add_artist(circle)

In [ ]:
sns.catplot(y='Type', x='Age', data=train_dataset.sort_values("Type", ascending=False),
           kind='boxen', height=6, aspect=3)

# ***Testing and Splitting Data***

In [ ]:
X = train_dataset.drop('Type', axis=1)
Y = train_dataset['Type']

In [ ]:
X.shape , Y.shape

In [ ]:
X.head(5)

In [ ]:
Y.head(5)

In [ ]:

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, 
                                                  test_size=0.2, 
                                                  random_state=123)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)
(1157, 23)
(290, 23)
(1157,)
(290,)

In [ ]:
X_arr = np.array(X)
X_arr.shape, X.shape

In [ ]:
X_arr

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

In [ ]:
labelled_Y = to_categorical(encoded_Y)

In [ ]:
n_classes = len(np.unique(Y))

In [ ]:
labelled_Y.shape

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                  labelled_Y, 
                                                  test_size=0.2, 
                                                  random_state=123)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

# ***Models***

****DNN****

In [ ]:
def naural_natwork():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(X_arr.shape[1],)))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    return model

In [ ]:
model= naural_natwork()

model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])


callback = EarlyStopping(monitor='val_accuracy', 
                                   mode='max', # don't minimize the accuracy!
                                   patience=10,
                                   restore_best_weights=True)



model.fit(X_train, Y_train, 
          epochs=100, 
          validation_data=(X_test, Y_test), verbose=1)

In [ ]:
model.summary()

In [ ]:
hist= model.history.history

In [ ]:
hist.keys()

In [ ]:
acc, val_acc = hist['accuracy'], hist['val_accuracy']
loss = hist['loss']
val_loss = hist['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy (Deep Neural Network V1)')

print('')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])

plt.legend(loc='upper right')
plt.title('Training and Validation Loss(Deep Neural Network V2)')
plt.xlabel('epoch')
plt.show()

In [ ]:
def neural_network_2():
    model = Sequential()
    model.add(Dense(256, activation='relu', input_shape=(X_arr.shape[1],)))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='softmax'))
    return model

In [ ]:
model_2 = neural_network_2()
model_2.summary()

In [ ]:
model_2.compile(optimizer='adam',
                loss='categorical_crossentropy', 
              metrics=['accuracy'])

# setting a callback for controlling model's performance
callback = EarlyStopping(monitor='val_accuracy', 
                                   mode='max',
                                   patience=10,
                                   restore_best_weights=True)


# fitting the model to training data
model_2.fit(X_train, Y_train, 
          epochs=100, 
          validation_data=(X_test, Y_test), 
          verbose=1)

In [ ]:
hist_2 = model_2.history.history

In [ ]:
acc_2, val_acc_2 = hist_2['accuracy'], hist_2['val_accuracy']
loss_2 = hist_2['loss']
val_loss_2 = hist_2['val_loss']

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc_2, label='Training Accuracy of DNN 2')
plt.plot(val_acc_2, label='Validation Accuracy of DNN 2')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy (Deep Neural Network V2)')

print('')

plt.subplot(2, 1, 2)
plt.plot(loss_2, label='Training Loss of DNN 2')
plt.plot(val_loss_2, label='Validation Loss of DNN 2')
plt.ylim([0, 1.0])

plt.legend(loc='upper right')
plt.title('Training and Validation Loss (Deep Neural Network V2)')
plt.xlabel('epoch')
plt.show()

In [ ]:
# evaluate the model
score = model.evaluate(X_test, Y_test)
score_2 = model_2.evaluate(X_test, Y_test)

# print accuracies of models 
print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))
print('Accuracy of First Deep Neural Network: %.2f' % (score[1]*100))
print('Loss of First Deep Neural Network: %.4f' % (score[0]))

print('')

print("%s: %.2f%%" % (model_2.metrics_names[1], score_2[1] * 100))
print('Accuracy of Second Deep Neural Network: %.2f' % (score_2[1]*100))
print('Loss of Second Deep Neural Network: %.4f' % (score_2[0]))

In [ ]:
# evaluate the model
score = model.evaluate(X_test, Y_test)
score_2 = model_2.evaluate(X_test, Y_test)

# print accuracies of models 
print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))
print('Accuracy of First Deep Neural Network: %.2f' % (score[1]*100))
print('Loss of First Deep Neural Network: %.4f' % (score[0]))

print('')

print("%s: %.2f%%" % (model_2.metrics_names[1], score_2[1] * 100))
print('Accuracy of Second Deep Neural Network: %.2f' % (score_2[1]*100))
print('Loss of Second Deep Neural Network: %.4f' % (score_2[0]))

In [ ]:
y_hat_nn1 = model.predict(X_test)
pred_nn1 = np.argmax(model.predict(X_test), axis=1)
Y_test_class_1 = np.argmax(Y_test, axis=1)

In [ ]:
print('Classification Report of Deep Neural Network 1')
print(classification_report(Y_test_class_1, pred_nn1))

In [ ]:
# classification report of second neural network
y_hat_nn2 = model_2.predict(X_test)
pred_nn2 = np.argmax(model_2.predict(X_test), axis=1)
Y_test_class_2 = np.argmax(Y_test, axis=1)
from sklearn.metrics import precision_recall_curve
predicted_probabilities = model_2.predict(X_test)
true_labels = np.argmax(Y_test, axis=1)

In [ ]:
from sklearn.metrics import precision_recall_curve, auc
precision = dict()
recall = dict()
pr_auc = dict()
n_classes = len(np.unique(true_labels))
for i in range(n_classes):
    precision[i], recall[i], _ = precision_recall_curve(
        (true_labels == i).astype(int), predicted_probabilities[:, i])
    pr_auc[i] = auc(recall[i], precision[i])
    precision["micro"], recall["micro"], _ = precision_recall_curve(
    np.array(Y_test).ravel(), predicted_probabilities.ravel())
pr_auc["micro"] = auc(recall["micro"], precision["micro"])

In [ ]:
print('Classification Report of Deep Neural Network 2')
print(classification_report(Y_test_class_2, pred_nn2))

In [ ]:
DNN_Accuracy = score_2[1]
DNN_Precision = precision_score(Y_test_class_2, pred_nn2,average = "macro")
DNN_Recall = recall_score(Y_test_class_2, pred_nn2,average = "macro")
DNN_F1 = f1_score(Y_test_class_2, pred_nn2,average = "macro")
print('accuracy',DNN_Accuracy)
print("Precision Score : ",DNN_Precision)
print("Recall Score :" , DNN_Recall)
print("F1 Score :" , DNN_F1)

In [ ]:
from sklearn.metrics import roc_curve, auc

In [ ]:
n_classes = Y_train.shape[1]
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(Y_test[:, i], y_hat_nn2[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

In [ ]:
fpr["macro"], tpr["macro"], _ = roc_curve(Y_test.ravel(), y_hat_nn2.ravel())
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

In [ ]:
plt.figure()
lw = 2
plt.plot(fpr["macro"], tpr["macro"], color='deeppink',
         lw=lw, label='Macro-average ROC curve (area = {0:0.2f})'.format(roc_auc["macro"]))
for i in range(n_classes):
    plt.plot(fpr[i], tpr[i], lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve for DNN')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# Plot the PR curves for each class
plt.figure()
plt.step(recall['micro'], precision['micro'], color='b', alpha=0.2,
         where='post', label='Micro-average (AUC = {0:0.2f})'
               ''.format(pr_auc["micro"]))
for i in range(n_classes):
    plt.step(recall[i], precision[i], where='post',
             label='Class {0} (AUC = {1:0.2f})'
             ''.format(i, pr_auc[i]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for DNN')
plt.legend(loc='lower left')
plt.grid(True)
plt.show()

In [ ]:
x=df.drop(['Type'],1)
y=df['Type'].copy()

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(x,y, test_size = 0.2, random_state = 101)

In [ ]:
scaler=StandardScaler()
X_train_scale=scaler.fit_transform(xtrain)
X_test_scale = scaler.transform(xtest)

 ****KNN****

In [ ]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import precision_recall_curve, average_precision_score
from itertools import cycle
from sklearn.metrics import roc_curve

kRange = range(1,15,1)
scores_list = []
pr_curve_dict = {}
for i in kRange:
    knn =  KNeighborsClassifier(n_neighbors = i)
    knn.fit(X_train_scale,ytrain)
    ypredr = knn.predict(X_test_scale)
Acc_KNN=accuracy_score(ypredr, ytest)
Pre_KNN = precision_score(ytest, ypredr,average = "macro")
Rec_KNN = recall_score(ytest, ypredr, average = "macro")
F1_KNN = f1_score(ytest, ypredr, average = "macro")
print('accuracy',Acc_KNN)
print("Precision Score : ",Pre_KNN)
print("Recall Score :" , Rec_KNN)
print("F1 Score :" , F1_KNN )

y_scores = knn.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for KNN')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = knn.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for KNN')
plt.legend()
plt.grid(True)
plt.show()

****DecisionTreeClassifier****

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
dtr = DecisionTreeClassifier(random_state=32)
dtr.fit(X_train_scale, ytrain)
ypreddtf = dtr.predict(X_test_scale)
Acc_DTC=accuracy_score(ypreddtf, ytest)
Pre_DTC = precision_score(ytest, ypreddtf ,average = "macro")
Rec_DTC = recall_score(ytest, ypreddtf , average = "macro")
F1_DTC = f1_score(ytest, ypreddtf , average = "macro")
print('accuracy',Acc_DTC)
print("Precision Score : ",Pre_DTC)
print("Recall Score :" , Rec_DTC)
print("F1 Score :" , F1_DTC)
y_scores = dtr.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for Decision Tree Classifier')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = dtr.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Decision Tree Classifier')
plt.legend()
plt.grid(True)
plt.show()

****MLPClassifier****

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
mlp=MLPClassifier()
mlp.fit(X_train_scale, ytrain)
ypredmlp = mlp.predict(X_test_scale)
Acc_mlp=accuracy_score(ypredmlp,ytest )
Pre_mlp = precision_score(ytest, ypredmlp ,average = "macro")
Rec_mlp = recall_score(ytest, ypredmlp , average = "macro")
F1_mlp = f1_score(ytest, ypredmlp , average = "macro")
print('accuracy',Acc_mlp)
print("Precision Score : ",Pre_mlp)
print("Recall Score :" ,Rec_mlp )
print("F1 Score :" ,F1_mlp )

y_scores = mlp.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for MLPClassifier')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = mlp.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for MLPClassifier')
plt.legend()
plt.grid(True)
plt.show()

****SVM****

In [ ]:
from sklearn import svm
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
clf = svm.SVC(kernel='linear', C = 1.0, probability=True)
clf.fit(X_train_scale, ytrain)
ypredclf = clf.predict(X_test_scale)
Acc_clf=accuracy_score(ytest,ypredclf)
Pre_clf = precision_score(ytest, ypredclf ,average = "macro")
Rec_clf = recall_score(ytest, ypredclf , average = "macro")
F1_clf = f1_score(ytest, ypredclf , average = "macro")
print('accuracy',Acc_clf)
print("Precision Score : ",Pre_clf)
print("Recall Score :" , Rec_clf)
print("F1 Score :" , F1_clf)

y_scores = clf.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for SVM')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = clf.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx] , y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for SVM')
plt.legend()
plt.grid(True)
plt.show()

****Random Forest****

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
rfr = RandomForestClassifier(random_state= 32)
rfr.fit(X_train_scale, ytrain)
ypredrfr = rfr.predict(X_test_scale)
Acc_RF=accuracy_score(ypredrfr,ytest)
Pre_RF = precision_score(ytest, ypredrfr ,average = "macro")
Rec_RF = recall_score(ytest, ypredrfr , average = "macro")
F1_RF = f1_score(ytest, ypredrfr , average = "macro")
print('accuracy',Acc_RF)
print("Precision Score : ",Pre_RF)
print("Recall Score :" , Rec_RF)
print("F1 Score :" , F1_RF)

y_scores = rfr.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for Random Forest')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = rfr.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for Random Forest')
plt.legend()
plt.grid(True)
plt.show()

****MLP With GridSearch****

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
param_grid = {
        'hidden_layer_sizes': [(7, 7), (128,), (128, 7)],
        'tol': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6],
        'epsilon': [1e-3, 1e-7, 1e-8, 1e-9, 1e-8]
    }
estimator = GridSearchCV(
        MLPClassifier(learning_rate='adaptive', learning_rate_init=1., early_stopping=True, shuffle=True),
        param_grid=param_grid, n_jobs=-1)
estimator.fit(X_train_scale, ytrain)
grid_predictions = estimator.predict(X_test_scale)
Acc_grid_mlp=accuracy_score(grid_predictions,ytest)
Pre_grid_mlp=precision_score(ytest, grid_predictions ,average = "macro")
Rec_grid_mlp=recall_score(ytest, grid_predictions , average = "macro")
F1_grid_mlp=f1_score(ytest, grid_predictions , average = "macro")
print('accuracy',Acc_grid_mlp)
print("Precision Score : ",Pre_grid_mlp)
print("Recall Score :" , Rec_grid_mlp)
print("F1 Score :" , F1_grid_mlp)
y_scores = estimator.predict_proba(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], y_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), y_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for MLP With GridSearch')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = estimator.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for MLP With GridSearch')
plt.legend()
plt.grid(True)
plt.show()

****SVC With Gridsearch****

In [ ]:
from sklearn.metrics import precision_recall_curve, average_precision_score, auc
param_grid = {'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001]} 
# grida = GridSearchCV(svm.SVC(),param_grid,refit=True,verbose=2)
grida = GridSearchCV(svm.SVC(kernel='rbf', probability=True), param_grid, refit=True, verbose=2)
grida.fit(X_train_scale,ytrain)
grid_predictions = grida.predict(X_test_scale)
Acc_grid_scv=accuracy_score(grid_predictions,ytest)
Pre_grid_scv = precision_score(ytest, grid_predictions ,average = "macro")
Rec_grid_scv = recall_score(ytest, grid_predictions , average = "macro")
F1_grid_scv = f1_score(ytest, grid_predictions , average = "macro")
print('accuracy',Acc_grid_scv)
print("Precision Score : ",Pre_grid_scv)
print("Recall Score :" , Rec_grid_scv)
print("F1 Score :" , F1_grid_scv)

decision_scores = grida.decision_function(X_test_scale)
y_test_bin = label_binarize(ytest, classes=np.unique(ytrain))
fpr = dict()
tpr = dict()
auc_score = dict()
n_classes = y_test_bin.shape[1]
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test_bin[:, i], decision_scores[:, i])
    auc_score[i] = auc(fpr[i], tpr[i])
    fpr["micro"], tpr["micro"], _ = roc_curve(y_test_bin.ravel(), decision_scores.ravel())
auc_score["micro"] = auc(fpr["micro"], tpr["micro"])
plt.figure()
colors = cycle(['blue', 'red', 'green', 'purple'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=2,
             label='ROC curve (AUC = {:.2f}) for class {}'.format(auc_score[i], i))
plt.plot(fpr["micro"], tpr["micro"],
         label='Micro-average ROC curve (AUC = {:.2f})'.format(auc_score["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot([0, 1], [0, 1], 'k--')  # diagonal line representing random guessing
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate (FPR)')
plt.ylabel('True Positive Rate (TPR)')
plt.title('ROC Curve for SVC With Gridsearch')
plt.legend(loc='lower right')
plt.show()

In [ ]:
    pr_curve_dict = {}
    precision = dict()
    recall = dict()
    average_precision = dict()
    y_scores = grida.predict_proba(X_test_scale)
    
    for class_idx in range(len(np.unique(ytrain))):
        precision[class_idx], recall[class_idx], _ = precision_recall_curve(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
        average_precision[class_idx] = average_precision_score(
            y_test_bin[:, class_idx], y_scores[:, class_idx])
    
    scores_list.append(average_precision)
    pr_curve_dict[i] = {'precision': precision, 'recall': recall}

for class_idx in range(len(np.unique(ytrain))):
     auc_scores = []
    for k, pr_curve in pr_curve_dict.items():
        auc = average_precision[class_idx]
        auc_scores.append(auc)
    print("Class {}: Mean AUC = {:.2f}".format(class_idx, np.mean(auc_scores)))
    # Plot precision-recall curves for each class
plt.figure(figsize=(8, 6))

for class_idx in range(len(np.unique(ytrain))):
     for k, pr_curve in pr_curve_dict.items():
        precision = pr_curve['precision'][class_idx]
        recall = pr_curve['recall'][class_idx]
        plt.plot(recall, precision, label='Class {} - AUC={:.2f}'.format(class_idx, average_precision[class_idx]))

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curve for SVC With Gridsearch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'MLPClassifier', 
              'KNeighborsClassifier', 'DecisionTreeClassifier','support vector classifier','svc with gridsearch','mlp with gridsearch','DNN'] ,
    'Accuracy Score': [Acc_RF, Acc_mlp, 
              Acc_KNN, Acc_DTC,Acc_clf,Acc_grid_scv,Acc_grid_mlp,DNN_Accuracy ]})
models.sort_values(by='Accuracy Score', ascending=False)

In [ ]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'MLPClassifier', 
              'KNeighborsClassifier', 'DecisionTreeClassifier','support vector classifier','svc with gridsearch','mlp with gridsearch','DNN'] ,
    'Precision Score': [Pre_RF, Pre_mlp, 
              Pre_KNN, Pre_DTC,Pre_clf,Pre_grid_scv,Pre_grid_mlp,DNN_Precision ]})
models.sort_values(by='Precision Score', ascending=False)

In [ ]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'MLPClassifier', 
              'KNeighborsClassifier', 'DecisionTreeClassifier','support vector classifier','svc with gridsearch','mlp with gridsearch','DNN'] ,
    'Recall Score': [Rec_RF, Rec_mlp, 
              Rec_KNN, Rec_DTC,Rec_clf,Rec_grid_scv,Rec_grid_mlp,DNN_Recall ]})
models.sort_values(by='Recall Score', ascending=False)

In [ ]:
models = pd.DataFrame({
    'Model': ['Random Forest', 'MLPClassifier', 
              'KNeighborsClassifier', 'DecisionTreeClassifier','support vector classifier','svc with gridsearch','mlp with gridsearch','DNN'] ,
    'F1 Score': [F1_RF, F1_mlp, 
              F1_KNN, F1_DTC,F1_clf,F1_grid_scv,F1_grid_mlp,DNN_F1 ]})
models.sort_values(by='F1 Score', ascending=False)